# Crunched Connection

## This tool uses flight records, archived weather forecasts, and supervised machine learning to determine likelihood of missing an upcoming flight connection. Flight data from U.S. Bureau of Transportation Statistics, weather data from National Weather Service Archives

### Prepare merged flight and weather dataset so it can be used as input in ML modeling

In [ ]:
#Import modules
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
import pickle

In [ ]:
#Read merged flight+met data for ML and perform slight processing
file2019 = '../data/processed/merged/2019_FlightMetMerged.csv'
df2019 = pd.read_csv(file2019)

file2018 = '../data/processed/merged/2018_FlightMetMerged.csv'
df2018 = pd.read_csv(file2018)

file2017 = '../data/processed/merged/2017_FlightMetMerged.csv'
df2017 = pd.read_csv(file2017)

In [ ]:
#Combine 2017, 2018, 2019
dftmp = [df2017,df2018,df2019]
df = pd.concat(dftmp)

In [ ]:
#Drop any duplicate rows
df.drop_duplicates(inplace=True)

In [ ]:
#Drop rows where cancellations are not due to weather
dfcc = df[ (df['CANCELLATION_CODE']!='A') & (df['CANCELLATION_CODE']!='C') & (df['CANCELLATION_CODE']!='D') ]

In [ ]:
#Prevent exclusion of cancelled flights due to weather
dfcc.loc[(dfcc['CANCELLATION_CODE']=='B'),'ARR_DELAY_GROUP'] = 12

In [ ]:
#Drop rows where there is a delay that isn't at least partially due to severe weather
dfccOnlyWxDelay = dfcc[~(dfcc['WEATHER_DELAY']==0)]

In [ ]:
#Make sure flights cancelled from weather are still in dataset
dfccOnlyWxDelay.loc[(dfccOnlyWxDelay['CANCELLATION_CODE']=='B'),'ARR_DELAY_GROUP'] = 12

In [ ]:
#Drop cancellation, diversion, and delay information columns from dataset
dfccOnlyWxDelay.drop(columns=['CANCELLED','DIVERTED','WEATHER_DELAY','NAS_DELAY','LATE_AIRCRAFT_DELAY','CANCELLATION_CODE'],axis=1,inplace=True)

In [ ]:
#Drop any rows with missing values
dfML = dfccOnlyWxDelay.dropna() 

In [ ]:
#Merge arrival bins for binary classification

#Less than 15 minutes late (which is considered on-time)
dfML['ARR_DELAY_GROUP'].loc[(dfML['ARR_DELAY_GROUP']<=0)] = 0

#More than 15 minutes late
dfML['ARR_DELAY_GROUP'].loc[(dfML['ARR_DELAY_GROUP']>0)] = 1

In [ ]:
#Dataframe name change 
dfMLmerged = dfML.copy()

### Feature engineering and selection

In [ ]:
#Label encode categorical variable
dfMLmerged['OP_UNIQUE_CARRIER'].replace({'MQ':1,'AA':2,'B6':3,'OO':4,'OH':5,'UA':6,'AS':7,'NK':8,
                                 'YX':9,'YV':10,'WN':11,'DL':12,'EV':13,'9E':14,'F9':15,'HA':16,'G4':17,'VX':18},inplace=True)

In [ ]:
#Experiment with features based on time of flight
dfMLmerged['DEP_TIME'] = pd.to_datetime(dfMLmerged['DEP_TIME'])
dfMLmerged['ARR_TIME'] = pd.to_datetime(dfMLmerged['ARR_TIME'])

#Convert date infomation to feature info
dfMLmerged.loc[(dfMLmerged['DEP_TIME'].dt.month==12) | (dfMLmerged['DEP_TIME'].dt.month==1) | (dfMLmerged['DEP_TIME'].dt.month==2),'SEASON'] = 1
dfMLmerged.loc[(dfMLmerged['DEP_TIME'].dt.month==3) | (dfMLmerged['DEP_TIME'].dt.month==4) | (dfMLmerged['DEP_TIME'].dt.month==5),'SEASON'] = 2
dfMLmerged.loc[(dfMLmerged['DEP_TIME'].dt.month==6) | (dfMLmerged['DEP_TIME'].dt.month==7) | (dfMLmerged['DEP_TIME'].dt.month==8),'SEASON'] = 3
dfMLmerged.loc[(dfMLmerged['DEP_TIME'].dt.month==9) | (dfMLmerged['DEP_TIME'].dt.month==10) | (dfMLmerged['DEP_TIME'].dt.month==11),'SEASON'] = 4

dfMLmerged.loc[(dfMLmerged['DEP_TIME'].dt.hour>=1) | (dfMLmerged['DEP_TIME'].dt.hour<=8),'TOD'] = 1
dfMLmerged.loc[(dfMLmerged['DEP_TIME'].dt.hour>=9) | (dfMLmerged['DEP_TIME'].dt.hour<=14),'TOD'] = 2
dfMLmerged.loc[(dfMLmerged['DEP_TIME'].dt.hour>=15) | (dfMLmerged['DEP_TIME'].dt.hour<=23),'TOD'] = 3

dfMLmerged['month'] = dfMLmerged['DEP_TIME'].dt.month
dfMLmerged['hour'] = dfMLmerged['DEP_TIME'].dt.hour
dfMLmerged['DOW'] = dfMLmerged['DEP_TIME'].dt.dayofweek
dfMLmerged['DOY'] = dfMLmerged['DEP_TIME'].dt.dayofyear

In [ ]:
#Specify which features to include in ML model
X = dfMLmerged[['spd_D', 'fzRnPrb_D', 'snowPrb_D', '6hrTsPrb_15mi_D',
       '6hrSvrTsPrb_25mi_D', 'fzRnPrb_A', 'snowPrb_A', '6hrTsPrb_15mi_A',
       '6hrSvrTsPrb_25mi_A', '6hPrecPrb_D', '6hPrecPrb_A', '12hPrecPrb_D',
       '12hPrecPrb_A', 'snow_D', 'snow_A', 'spd_A', 'tmpF_D', 'dptF_D',
       'tmpF_A', 'dptF_A', 'ceil_D', 'ceil_A', 'visib_D', 'visib_A',
       '6hQntPrec_D', '6hQntPrec_A', '12hQntPrec_A', '12hQntPrec_D',
       'OP_UNIQUE_CARRIER', 'DISTANCE_GROUP', 'month', 'hour', 'DOY']]

y = dfMLmerged['ARR_DELAY_GROUP']

### Split dataset into training and test set, then balance the training dataset

In [ ]:
#Split conversion dataset into train and test groups
X_train, X_test, y_train, y_test = train_test_split(X, y,stratify=y,shuffle=True,random_state=42)

In [ ]:
#Balance the dataset
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler

# define oversampling strategy
over = RandomOverSampler(sampling_strategy='minority',random_state=42)

# fit and apply the transform
X_train_over, y_train_over = over.fit_resample(X_train, y_train)

# define understampling strategt
under = RandomUnderSampler(sampling_strategy='majority',random_state=42)

# fit and apply the transform
X_train_under, y_train_under = under.fit_resample(X_train_over, y_train_over)

### Train and test machine learning model

In [ ]:
#Train random forest model
#clf = RandomForestClassifier(n_estimators=20,n_jobs=5,class_weight='balanced',random_state=42,max_features=10).fit(X_train_under, y_train_under)
clf = RandomForestClassifier(n_estimators=10,n_jobs=5,class_weight='balanced',random_state=42,max_features=10).fit(X_train_under, y_train_under)

In [ ]:
#Save trained model to a pickle file
pkl_filename = "pickle_model.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(clf, file)

In [ ]:
#Try out XGBoost and sklearn gradient boosting ML models

#from xgboost import XGBClassifier
#model_xgb = XGBClassifier(random_state=42,n_estimators=10).fit(X_train_under, y_train_under)

#from sklearn.ensemble import GradientBoostingClassifier
#gb_clf = GradientBoostingClassifier(n_estimators=20,random_state=42).fit(X_train_under, y_train_under)

In [ ]:
#Run trained model on test set

forest_predicted = clf.predict(X_test)
#forest_predicted = gb_clf.predict(X_test)
#forest_predicted = model_xgb.predict(X_test)

### Model Performance Evaluation

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import classification_report
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import cohen_kappa_score

print('Classification Report')
print(classification_report(y_test, forest_predicted,target_names=['0', '1']))

print (' ')

print('Micro-averaged precision = {:.2f} (treat instances equally)'
       .format(precision_score(y_test, forest_predicted, average = 'micro')))
print('Macro-averaged precision = {:.2f} (treat classes equally)'
      .format(precision_score(y_test, forest_predicted, average = 'macro')))
print('Micro-averaged f1 = {:.2f} (treat instances equally)'
      .format(f1_score(y_test, forest_predicted, average = 'micro')))
print('Macro-averaged f1 = {:.2f} (treat classes equally)'
      .format(f1_score(y_test, forest_predicted, average = 'macro')))

print (' ')

print('Matthews correlation coefficient = {:.2f}'
      .format(matthews_corrcoef(y_test, forest_predicted)))

print('Cohen kappa score = {:.2f}'
      .format(cohen_kappa_score(y_test, forest_predicted)))

print (' ')
print ('Confusion Matrix')
confusion_matrix(y_test, forest_predicted)

In [ ]:
#Examine feature importance
features = X
feature_list = list(features.columns)

#importances = list(model_xgb.feature_importances_)
#importances = list(gb_clf.feature_importances_)
importances = list(clf.feature_importances_)

# List of tuples with variable and importance
feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(feature_list, importances)]
# Sort the feature importances by most important first
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)
# Print out the feature and importances 
[print('Variable: {:12} Importance: {}'.format(*pair)) for pair in feature_importances]

In [ ]:
#Predict probabilities associated with test set
#predictions = clf.predict_proba(X_test)